In [6]:
import joblib
import os
import itertools
import re
import unidecode
import string
import cmath

import numpy as np
import pandas as pd

from zipfile import ZipFile
from collections import defaultdict
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# Functions

In [7]:
def create_ngram_base(alphabet, n_order):
    assert type(alphabet) == list, 'Alphabet is not list.'
    assert len(set(alphabet)) == len(alphabet), 'Alphabet is not correct.'
    alphabet.sort()
    n_gram_base = set()
    for combination in itertools.combinations_with_replacement(alphabet, n_order):
        for permutation in itertools.permutations(combination, n_order):
            n_gram = ''
            for letter in permutation:
                n_gram += letter
            n_gram_base.add(n_gram)
    assert len(n_gram_base) == len(alphabet)**n_order, 'Incorrect result'
    print("Created", str(n_order)+"-gram base with", len(n_gram_base), str(n_order)+"-grams.")
    n_gram_base = list(n_gram_base)
    n_gram_base.sort()
    return n_gram_base

def alphabet():
    alphabet = list(string.ascii_lowercase)
    alphabet.append(' ')
    return alphabet

def process_sentenses(path_to_file):
    alph = alphabet()
    delimiters = "\t", "\n"
    regexPattern = '|'.join(map(re.escape, delimiters))
    regexPattern

    raw_lines = []
    with open(path_to_file, 'r', encoding="utf-8") as file:
        for line in file:
            raw_lines.append(line)
        
    lines_split = [re.split(regexPattern, line)[1].lower() for line in raw_lines]
    
    lines_final = []

    for i in tqdm(range(len(lines_split))):
        line = lines_split[i]
        line_ = [unidecode.unidecode(x) for x in line if x.isalpha() or x.isspace()]
        
        line_ = [x for x in ''.join(line_).split(' ') if x]
        new_line = []
        for word in line_:
            word = [x for x in word if x in alph]
            new_line.append(''.join(word).lower())
        lines_final.append(new_line)
    del delimiters, regexPattern, raw_lines, lines_split, line, line_
    return lines_final

def create_blocks(lines):    
    blocks = []
    current_block = ''
    
    for i in tqdm(range(len(lines))):
        line = ' '.join(lines[i])
        if len(current_block) < 1000:
            if len(current_block) > 0:
                if current_block[-1] != ' ': current_block += ' '
            current_block += line
        elif len(current_block) >= 1000:
            blocks.append(current_block[:1000])
            current_block = line
    del current_block, line
    return blocks

def base_26_latin_ngrams(n):
    alb = alphabet()
    base_ngrams = create_ngram_base(alb, n)
    del alb
    return base_ngrams

def create_rand_atom_vect(order=1000):
    atom = np.random.randint(-1, 1, order)
    atom = np.where(atom==0, 1, atom)
    atom.reshape(-1,1)
    return atom

def create_item_memory(order=1000, alphabet_size=27):
    item_memory = []
    for i in range(alphabet_size):
        item_memory.append(create_rand_atom_vect(order))
    item_memory = np.array(item_memory)
    return item_memory    

def create_fixed_permutations(order=1000, n=3):
    index = [i for i in range(order)]
    permutations = []
    for i in range(n):
        np.random.shuffle(index)
        permutations.append(index.copy())
    return permutations

def bind(hdv1, hdv2):
    binding = hdv1*hdv2
    return binding
    
def permute(hdv, permutation, n_times_to_permute):
    for i in range(n_times_to_permute):
        hdv = hdv[permutation]
    return hdv
    
def bundle(hdv1, hdv2):
    bundling = hdv1 + hdv2
    return bundling

def measure_cosine_similarity(hdv1, hdv2):
    return(cosine_similarity([hdv1], [hdv2]))

def jobsave(object_to_save, filename):
    joblib.dump(object_to_save, filename)

def jobload(filename):
    return joblib.load(filename)

# Data

In [ ]:
train_data_root = os.path.join('data', 'train')
data_lang_paths = os.listdir(train_data_root)

In [ ]:
data_lang_paths

In [ ]:
lang_paths = defaultdict()
for path in data_lang_paths:
    lang_paths[path[:3]] = os.path.join(train_data_root, path, path+'-sentences.txt')

In [ ]:
lang_paths

In [ ]:
lines_raw = defaultdict()
for lang in lang_paths:
    lines_raw[lang] = process_sentenses(lang_paths[lang])

In [ ]:
lang_blocks = defaultdict()
for lang in lines_raw:
    lang_blocks[lang] = create_blocks(lines_raw[lang])

In [ ]:
# save the data
fname = 'lang_blocks_train.jbl'
jobsave(lang_blocks, fname)

In [ ]:
# load the data
fname = 'lang_blocks_train.jbl'
lang_blocks_train = jobload(fname)

# Experiment 1

# 1. Encode train data using HD vectors, 3-grams and 21 languages

## 1.1 create latin alphabet and item memory using 27 atom HD binary (-1, 1) vectors with size 1000

In [3]:
# create 26 letter + 1 space symbol alphabet:
latin_alphabet = alphabet()

In [ ]:
# create item memory
item_memory = create_item_memory(order=1000, alphabet_size=27)

# check the shape; sum of item memory can show if some bias exist in random destribution
item_memory.shape, np.sum(item_memory)

## 1.2 create fixed permutations 

In [ ]:
fixed_permutations = create_fixed_permutations(order=1000, n=3)

## 1.3 encode hd vector representations for all possible 3-grams

In [ ]:
base_ngrams = base_26_latin_ngrams(3)

In [ ]:
def encode_ngram(ngram):
    vectors = []
    for order in range(len(ngram)):
        letter = ngram[order]
        item_id = latin_alphabet.index(letter)
        
        # this is atom vector for that simbol
        item_itself = item_memory[item_id]
        permutation_n = fixed_permutations[order]
        permuted_hdv = permute(item_itself, permutation_n, 1)
        vectors.append(permuted_hdv)
    output_vector = vectors[0]
    for hdv in vectors[1:]:
        output_vector = bind(output_vector, hdv)
    return output_vector

In [ ]:
encoded_trigrams = defaultdict()
for trigram in base_ngrams:
    encoded_trigrams[trigram] = encode_ngram(trigram)

## Saving the intermediate results

In [ ]:
# save the results
jobsave(item_memory, 'item_memory.jbl')
jobsave(fixed_permutations, 'fixed_permutations.jbl')
jobsave(base_ngrams, 'base_ngrams.jbl')
jobsave(encoded_trigrams, 'encoded_trigrams.jbl')

In [ ]:
# load the results
item_memory = jobload('item_memory.jbl')
fixed_permutations = jobload('fixed_permutations.jbl')
base_ngrams = jobload('base_ngrams.jbl')
encoded_trigrams = jobload('encoded_trigrams.jbl')

# 1.4 Unsing encoded n-grams, encode the training data blocks

In [10]:
def parse_encode_text_block(data_block, encoded_trigrams):

    tri_grams = [data_block[i:i+3] for i in range(len(data_block)-2)]
    encoded_block_trigrams = []
    for tg in tri_grams:
        encoded_block_trigrams.append(encoded_trigrams[tg.lower()])
    
    enc_tg_start = encoded_block_trigrams[0]
    for enc_tg in encoded_block_trigrams:
        enc_tg_start = bundle(enc_tg_start, enc_tg)
    return enc_tg_start

In [ ]:
encoded_lang_blocks_train = defaultdict()
for language in lang_blocks_train:
    print(language)
    encoded_list = parse_encode_text_block(lang_blocks_train[language][0], encoded_trigrams)
    for b in tqdm(range(1, len(lang_blocks_train[language]))):
        #print(b)
        block = lang_blocks_train[language][b]
        encoded_list = bundle(encoded_list , parse_encode_text_block(block, encoded_trigrams))
    encoded_lang_blocks_train[language] = encoded_list

In [ ]:
jobsave(encoded_lang_blocks_train, 'encoded_lang_blocks_train.jbl')

In [ ]:
encoded_lang_blocks_train['bul'][1052]

# Create test data

In [ ]:
def process_sentenses_eurpar(path_to_file):
    alph = alphabet()
    
    _lines = []
    
    with open(path_to_file, 'r', encoding="utf-8") as file:
        for line in file:
            new_line = ''.join([x for x in unidecode.unidecode(line[:-1].lower()) if x in latin_alphabet])
            if len(new_line) > 0:
                _lines.append(' '.join(new_line.split()))
                
    return _lines

def process_sentenses_nn(path_to_file, nn):
    alph = alphabet()
    
    _lines = []
    
    with open(path_to_file, 'r', encoding="utf-8") as file:
        counter = 0
        for line in file:
            if counter <nn+1:
                new_line = ''.join([x for x in unidecode.unidecode(line[:-1].lower()) if x in latin_alphabet])
                if len(new_line) > 0:
                    _lines.append(' '.join(new_line.split()))
                    counter+=1
            else: 
                break

    return _lines

def create_blocks_eurpar(lines):    
    blocks = []
    for i in tqdm(range(len(lines))):
        line = ' '.join(lines[i])
        blocks.append(line)
    del line
    return blocks

In [ ]:
test_data_root = os.path.join('data', 'test')
data_lang_paths = os.listdir(test_data_root)

In [ ]:
data_lang_paths

In [ ]:
lang_paths_test = defaultdict()
for path in data_lang_paths:
    lang_paths_test[path[:2]] = os.path.join(test_data_root, path, 'europarl-v7.'+path+'.'+path[:2])

In [ ]:
lang_paths_test

In [ ]:
lines_test = defaultdict()
for lang in lang_paths_test:
    print(lang)
    lines_test[lang] = process_sentenses_nn(lang_paths_test[lang], 3000)
    
jobsave(lines_test, 'lines_test_nn.jbl')

In [ ]:
lines_test = defaultdict()
for lang in lang_paths_test:
    print(lang)
    lines_test[lang] = process_sentenses_eurpar(lang_paths_test[lang])
    
jobsave(lines_test, 'lines_test_1000.jbl')

In [ ]:
lines_test = jobload('lines_test.jbl')
lines_test_2 = jobload('lines_test_2.jbl')

In [ ]:
def parse_encode_text_block(data_block, encoded_trigrams):

    tri_grams = [data_block[i:i+3] for i in range(len(data_block)-2)]
    encoded_block_trigrams = []
    for tg in tri_grams:
        encoded_block_trigrams.append(encoded_trigrams[tg])
    
    enc_tg_start = encoded_block_trigrams[0]
    for enc_tg in encoded_block_trigrams:
        enc_tg_start = bundle(enc_tg_start, enc_tg)
    return enc_tg_start

In [ ]:
normalize([aaa], norm='l2')

In [ ]:
lines_test_nn = jobload('lines_test_nn.jbl')

In [ ]:
encoded_trigrams['aaa']

In [ ]:
#encoded_blocks_test = defaultdict()
for language in lines_test_nn:
    print(language)
    encoded_list = []
    for bl in tqdm(range(len(lines_test_nn[language]))):
        block = lines_test[language][bl]
        if len(block) >= 3:
            encoded_list.append(normalize([parse_encode_text_block(block, encoded_trigrams)], norm='l2'))
    root = 'encoded_langs_test_2'
    filename = language +'_encoded_test.jbl'
    path = os.path.join(root, filename)
    jobsave(encoded_list, path)
    del encoded_list

In [ ]:
#encoded_blocks_test = defaultdict()
for language in lines_test_2:
    print(language)
    encoded_list = []
    for bl in tqdm(range(len(lines_test_2[language]))):
        #if bl%10000 == 0:
        #    print(language, ':', bl)
        block = lines_test_2[language][bl]
        if len(block) >= 3:
            encoded_list.append(normalize([parse_encode_text_block(block, encoded_trigrams)], norm='l2'))
    filename = language +'_encoded_test.jbl'
    jobsave(encoded_list, filename)
    del encoded_list

In [ ]:
# save the results
jobsave(encoded_blocks_test, 'encoded_blocks_test.jbl')

In [ ]:
encoded_lang_blocks_train = jobload('encoded_lang_blocks_train.jbl')

In [ ]:
train_keys = {
    'bul' : 'Bulgarian',
    'ces' : 'Czech',
    'dan' : 'Danish',
    'deu' : 'German',
    'ell' : 'Greek',
    'eng' : 'English',
    'est' : 'Estonian',
    'fin' : 'Finnish',
    'fra' : 'French',
    'hun' : 'Hungarian',
    'ita' : 'Italian',
    'lav' : 'Latvian',
    'lit' : 'Lithuanian',
    'nld' : 'Dutch',
    'pol' : 'Polish',
    'por' : 'Portuguese',
    'ron' : 'Romanian',
    'slk' : 'Slovak',
    'slv' : 'Slovene',
    'spa' : 'Spanish',
    'swe' : 'Swedish'
}

In [ ]:
nor_lang_train = defaultdict()
for key in encoded_lang_blocks_train:
    new_key = train_keys[key]
    norm_vect = normalize([encoded_lang_blocks_train[key]], norm='l2')
    nor_lang_train[new_key] = norm_vect

In [ ]:
jobsave(nor_lang_train, 'nor_lang_train.jbl')

# complex vectors

In [ ]:
lines_test = jobload('lines_test.jbl')
lines_test_2 = jobload('lines_test_2.jbl')

In [17]:
def img_counterpart(real):
    img_pos = (1- real**2)**0.5
    
    return img_pos

def create_complex_range_u1(decimals):
    real_range = np.linspace(-1, 1, (10**decimals)*2+1)
    complex_range = []
    for x in real_range:
        y = img_counterpart(x)
        if y != 0:
            complex_range.append(complex(x, y))
            complex_range.append(complex(x, -y))
        elif y ==0:
            complex_range.append(complex(x, y))
    complex_range = np.array(complex_range)
    complex_range.sort()
    return complex_range

def create_random_complex_vector(source, order = 1000):
    atom_vector = np.random.choice(source, order)
    return atom_vector

def bind_comp(hdv1, hdv2):
    binding = hdv1*hdv2
    return binding
    
def permute_comp(hdv, place):
    np.roll(hdv, place)
    return hdv
    
def bundle_comp(hdv1, hdv2):
    bundling = hdv1 + hdv2
    return bundling

def comp_cos_sim(hdv1, hdv2):
    return(cosine_similarity([hdv1.real], [hdv2.real]))

def create_comp_item_memory(order=1000, alphabet_size=27):
    item_memory = []
    for i in range(alphabet_size):
        item_memory.append(create_random_complex_vector(complex_source, order))
    item_memory = np.array(item_memory)
    return item_memory    

In [ ]:
complex_source = create_complex_range_u1(6)
len(complex_source)

# Experiment 2

### prepare complex data

In [ ]:
latin_alphabet = alphabet()

In [18]:
base_ngrams = base_26_latin_ngrams(3)

Created 3-gram base with 19683 3-grams.


In [ ]:
complex_source = create_complex_range_u1(6)

In [ ]:
comp_item_memory = create_comp_item_memory(order=1000)

In [15]:
def encode_comp_ngram(ngram):
    vectors = []
    for order in range(len(ngram)):
        letter = ngram[order]
        item_id = latin_alphabet.index(letter)

        item_itself = comp_item_memory[item_id]

        permuted_hdv = permute_comp(item_itself, order)
        vectors.append(permuted_hdv)
    output_vector = vectors[0]
    for hdv in vectors[1:]:
        output_vector = bind(output_vector, hdv)
    return output_vector

In [19]:
comp_encoded_trigrams = defaultdict()
for trigram in base_ngrams:
    comp_encoded_trigrams[trigram] = encode_comp_ngram(trigram)

In [20]:
jobsave(comp_encoded_trigrams, 'comp_encoded_trigrams.jbl')

In [ ]:
# save the results
jobsave(comp_item_memory, 'comp_item_memory.jbl')
jobsave(comp_encoded_trigrams, 'comp_encoded_trigrams.jbl')

In [12]:
# load the results
comp_item_memory = jobload('comp_item_memory.jbl')
base_ngrams = jobload('base_ngrams.jbl')
comp_encoded_trigrams = jobload('encoded_trigrams.jbl')

In [ ]:
lang_blocks_train = jobload('lang_blocks_train.jbl')

### encode text blocks

In [ ]:
def parse_encode_text_block(data_block, encoded_trigrams):

    tri_grams = [data_block[i:i+3] for i in range(len(data_block)-2)]
    encoded_block_trigrams = []
    for tg in tri_grams:
        encoded_block_trigrams.append(encoded_trigrams[tg.lower()])
    
    enc_tg_start = encoded_block_trigrams[0]
    for enc_tg in encoded_block_trigrams:
        enc_tg_start = bundle(enc_tg_start, enc_tg)
    return enc_tg_start

In [ ]:
comp_encoded_lang_blocks_train = defaultdict()
for language in lang_blocks_train:
    new_language = train_keys[language]
    print(language)
    print(new_language)
    encoded_list = parse_encode_text_block(lang_blocks_train[language][0], comp_encoded_trigrams)
    for b in tqdm(range(1, len(lang_blocks_train[language]))):
        #print(b)
        block = lang_blocks_train[language][b]
        encoded_list = bundle(encoded_list , parse_encode_text_block(block, comp_encoded_trigrams))
    comp_encoded_lang_blocks_train[new_language] = encoded_list

In [ ]:
jobsave(comp_encoded_lang_blocks_train, 'comp_encoded_lang_blocks_train.jbl')

# create complex test data

In [8]:
lines_test = jobload('lines_test.jbl')
lines_test_2 = jobload('lines_test_2.jbl')

In [ ]:
comp_encoded_trigrams = jobload('comp_encoded_trigrams.jbl')

In [ ]:
#encoded_blocks_test = defaultdict()
for language in lines_test_2:
    print(language)
    encoded_list = []
    for bl in tqdm(range(len(lines_test_2[language]))):

        block = lines_test_2[language][bl]
        if len(block) >= 3:
            encoded_list.append(parse_encode_text_block(block, comp_encoded_trigrams))
    filename = language +'comp_encoded_test.jbl'
    root = 'comp_encoded_langs_test'
    jobsave(encoded_list, os.path.join(root, filename))
    del encoded_list

  0%|          | 403/1990304 [00:00<16:47, 1975.45it/s]

nl


  0%|          | 16/637649 [00:00<1:08:49, 154.41it/s]

sk


  0%|          | 1389/620457 [00:00<01:27, 7109.57it/s]

sl


  0%|          | 385/1852652 [00:00<18:47, 1643.28it/s]

sv


  6%|▌         | 113500/1852652 [00:41<11:03, 2621.71it/s]

In [24]:
#encoded_blocks_test = defaultdict()
for language in lines_test:
    print(language)
    encoded_list = []
    for bl in tqdm(range(len(lines_test[language]))):

        block = lines_test[language][bl]
        if len(block) >= 3:
            encoded_list.append(parse_encode_text_block(block, comp_encoded_trigrams))
    filename = language +'comp_encoded_test.jbl'
    root = 'comp_encoded_langs_test'
    jobsave(encoded_list, os.path.join(root, filename))
    del encoded_list

  0%|          | 121/404381 [00:00<05:35, 1206.34it/s]

bg


  0%|          | 831/643491 [00:00<01:17, 8294.13it/s]

cs


  0%|          | 251/1961219 [00:00<13:03, 2501.49it/s]

...

  0%|          | 245/1952785 [00:00<13:19, 2440.78it/s]

pt


  0%|          | 19/396797 [00:00<35:02, 188.70it/s]

ro


100%|██████████| 396797/396797 [02:51<00:00, 2315.24it/s]
